# <center>Estructura de Datos</center>
## <center>Universidad Nacional de Tres de Febrero</center>
## <center>TRZearcher</center>

### TRABAJO PRACTICO - TRANZAS DEL CODIGO
    Aguilera, Agustin
    Ambrosetti, Ramiro
    Noguera, Sol
    Tabarovsky, Javier

### TRZearcher
    Es un programa de busqueda y comparacion de precios de productos tecnologicos, para encontrar el mejor precio en el menor tiempo posible
    Por el momento trabajamos con: 
        - Compragamer
        - Full H4rd
        - Gezatek
        - Overdrive
        - Venex
    Estas paginas fueron seleccionadas debido a que son las mas economicas en su rubro a la fecha de publicacion de este informe, son paginas en continuo crecimiento, actualizadas diariamente, y con la mayor variedad de productos.
    
    Video presentacion: https://www.youtube.com/watch?v=i_qlmObgpmU&t=12s

### UML - Diagrama de clases

El UML esta en formato pdf para ahorrar espacio y subirlo a la plataforma

Puede encontrarse en este link para una mejor visualizacion:
    https://app.lucidchart.com/invitations/accept/5984dc3f-ae4b-4041-a64c-ddd82dfaeefc

### TECNICAS DE PROGRAMACION Y DISEÑO
    Las tecnicas de programacion utilizadas en este proyecto se basan en la programacion orientada a objetos, y la utilizacion de la libreria Scrapy.
    El diseño del programa esta dividido en 2 procesos, el Front y Back, esto se realizo de esta manera para solventar un problema.
    El problema se centraba en que al ejecutarse el programa, primero las spiders leian la informacion de configuracion y luego se ejecutaba la interfaz, esto ocasionaba que las url de inicio se encontraban incompletas
    El diseño se basa en la modularizacion de las funcionalidades, a continuacion se puede ver un diagrama donde explicamos como se desenvuelve el programa.

<img src="Ilustracion.png">

# El programa comienza su ejecucion desde la clase Main, la cual llama a Interface y la ejecuta:

    1 - Una vez que el usuario eligio su configuracion de busqueda, y presiona el boton de buscar, estas configuraciones son guardadas en un temporal denominado temp_connector, y otro temporal con las url a las paginas completas denominado pages_complete.
    2 - Este temp_conector es leido por la clase Proyect Connector.                           
    3 - La clase Proyect Connector, instancia a la clase Spider Connector la cual se encarga de :
        3.1 - Instanciar a la clase Orchestrator, la cual ejecuta las spiders seleccionadas, y estas mismas exportan los datos en temporales.csv.
        3.2 - Instanciar a la clase Sorter, la cual lee los temporales.csv, los ordena y filtra segun el tipo de busqueda, para luego instanciar a la clase Exporter con los resultados.
    4 - La clase Exporter se encarga de escribir los resultados en dos formatos, csv, json y html.
    Esta seria una explicacion a grandes rasgos del funcionamiento del programa         

### INTERFACE (FRONT)
    La interfaz esta creada como un proceso separado del programa, con una unica clase la cual crea una interfaz grafica con el uso de la libreria tkinter.
    En la cual a la hora de presionar el boton de buscar se realizan las siguientes acciones:
    
        1. Verifica que la entrada del producto no puede estar vacia, y al menos se debe seleccionar una pagina.

In [ ]:
    def __code_button(self):
            if (str(self.product.get()).strip() == "") and (self.compra_gamer.get() + self.full_hard.get() + self.gezatek.get() + self.venex.get() + self.overdrive.get() <= 0):
                messagebox.showerror(message="Recordar que:\n - La entrada del producto no puede estar vacia\n - Al menos hay que seleccionar una pagina", title="Problema con TRZearcher")
            else:
                self.__write_data_connector()
                self.__write_pages()
                subprocess.call("python ./Back/Connector/proyect_connector.py", shell=True)
                self.window.destroy()

        2. Escribe los datos de la interfaz en un temporal, llamado temp_connector.txt, el cual luego es leido por el otro proceso.

In [ ]:
    def __write_data_connector(self):
        with open("./Back/Connector/temp_connector.txt", 'w') as pages_file:
            pages_file.write(str(self.product.get()) + "\n")
            pages_file.write(str(self.compra_gamer.get()) + "\n")
            pages_file.write(str(self.full_hard.get()) + "\n")
            pages_file.write(str(self.gezatek.get()) + "\n")
            pages_file.write(str(self.venex.get()) + "\n")
            pages_file.write(str(self.overdrive.get()) + "\n")
            pages_file.write(str(self.option.get()) + "\n")

        3. Completa las paginas a buscar, leyendo de pages_initials.txt, agregando el producto al final y lo exporta a un temporal llamado page_complete.txt.

In [ ]:
    def __write_pages(self):
        pages_complete = []
        number_of_pages = 0
        fileInitials = os.path.abspath("./Back/Searcher/Data/pages_initials.txt")
        fileComplete = os.path.abspath("./Back/Searcher/Data/pages_complete.txt")
        with open(fileInitials, "r") as pages_file:
            for page in pages_file:
                number_of_pages += 1
                page = page.rstrip('\n') + str(self.product.get()) + "\n"
                pages_complete.append(page)
        with open(fileComplete, "w") as pages_file:
            for x in range(0, number_of_pages):
                pages_file.write(pages_complete[x])

        Una vez finalizado, se ejecuta el siguiente proceso, y se cierra la ventana.

### CONNECTOR (BACK)
    El modulo connector esta creado para la conexion del programa, la cual esta compuesta de dos clases proyect_connector y spyder_connector.
        1. proyect_connector: Se basa en la lectura de los datos obtenidos de interfaz (a travez del temporal temp_connector.txt).

In [ ]:
    __data = []
    with open("./Back/Connector/temp_connector.txt", "r") as data_connector:
        for data in data_connector:
            __data.append(str(data).strip())

    Connector(__data[0], __data[1], __data[2], __data[3], __data[4], __data[5], __data[6]).execute_connector()

        2. spyder_connector: Es la clase mas importante del programa, la cual es la encargada en primer instancia de verificar que paginas se quieren buscar, para proceder a ejecutar las Spiders, a su vez elimina los archivos temporales.

In [ ]:
    class Connector:

        def __init__(self, product, compragamer, fullhard, gezatek, venex, overdrive, type_search):
            self.product = product
            self.pages = self.__spiders_validation(compragamer, fullhard, gezatek, venex, overdrive)
            self.type_search = type_search

        def __spiders_validation(self, compragamer, fullhard, gezatek, venex, overdrive):
            pages = []
            if int(compragamer) == 1:
                pages.append("compragamer")
            if int(fullhard) == 1:
                pages.append("fullh4rd")
            if int(gezatek) == 1:
                pages.append("gezatek")
            if int(overdrive) == 1:
                pages.append("overdrive")
            if int(venex) == 1:
                pages.append("venex")
            return pages

        def execute_connector(self):
            Orchestrator().execute_spiders(self.pages)
            Sorter(self.pages, self.product, self.type_search).execute_sorter()
            self.__clean()
            sys.exit()

        def __clean(self):
            os.remove("./Back/Connector/temp_connector.txt")
            os.remove("./Back/Searcher/Data/pages_complete.txt")
            for page in self.pages:
                os.remove(page + ".csv")

### PROCESSOR (BACK)

El modulo processor tiene dos clases: Sorter y Exporter.
Este el encargado de ordenar y organizar los datos levantados de la web por el modulo searcher y escribirlos en distintos
formatos para la visualización de el resultado de la busqueda.

  ### Clase Sorter:

        Clase que a partir de archivos .csv de cada pagina con toda la informacion que el crawler levanto
        de la web, los lee y organiza para luego llamar a la clase Export, y escribir la salida.

Para hacer esto, se necesitan los inputs del usuario (lo que introdujo en la busqueda, las paginas en las que
desea buscar, y el tipo de busqueda), lo cual se ingresa en el constructor.


Este metodo es el que junta todos los pasos necesarios para cumplir el objetivo de la clase (ordenar y llamar a export):

In [ ]:
    def execute_sorter(self):
        self.__read_data()
        self.__sort_by_price(self.exact_words)
        self.__index_dict()
        self.__export()

El metodo read data lee los archivos .csv con categoria, precio, titulo y lo agrega a dos listas de
listas del tipo ["Page", "Category", "Title", "Price", "Link", "Time"]. Una lista exact_words contiene 
las que tienen titulo exactamente igual y not_exact_words el resto.

In [ ]:
   def __sort_by_price(self, products_list):
        products_list.sort(key=lambda e: e[3])

Ordena la lista de listas, itera y segun el elemento en la posicion 3 (que es el precio) lo reposiciona en la lista la cual se le pasa por parametro.

In [ ]:
    def __index_dict(self):
        self.__create_lists_in_dict()
        words = self.product_to_search.split()
        for product in self.not_exact_words:
            separated_title = product[2].split()
            counter = 0
            for word in words:
                if word in separated_title:
                    counter += 1

            self.matching_words_to_product[counter].append(product)

Crea el diccionario matching_words_to_product el cual indexa con clave de la cantidad de palabras
     que coinciden entre el titulo y lo que se busco. El valor son las listas de productos.

In [ ]:
    def __export(self):
        """
        Segun el tipo de busqueda elegida, se llamara a una instancia Export con los parametros correspondientes
        para que escriba el .csv con los datos ordenados.
        """

        separated_words = self.product_to_search.split()
        if self.search_type == "FRASE_EXACTA":
            exporter = Exporter(self.product_to_search, self.exact_words)
            self.__write(exporter)

        elif self.search_type == "CONTIENE_TODAS_LAS_PALABRAS":
            products_all_words = self.matching_words_to_product[len(separated_words)]
            self.__sort_by_price(products_all_words)
            products_all_words = self.exact_words + products_all_words
            exporter = Exporter(self.product_to_search, products_all_words)
            self.__write(exporter)

        elif self.search_type == "CONTIENE_ALGUNAS_PALABRAS":
            products_some_words = list()
            self.__collect_products(products_some_words, range(len(separated_words)))
            self.__sort_by_price(products_some_words)
            products_some_words = self.exact_words + products_some_words
            exporter = Exporter(self.product_to_search, products_some_words)
            self.__write(exporter)


   Este metodo se encarga de exportar los datos segun el tipo de busqueda elegida. Si se eligio frase exacta, simplemente
usa la lista exact_words para pasarsela a Export y escribir en csv json y html.

Si se eligio frase que contenga todas las palabras, busca en el diccionario que indexamos con anterioridad, la lista de productos que tienen la misma cantidad de coincidencias que numero de palabras se introdujo y la ordenamos. Luego, no podemos olvidarnos de las palabras exactamente iguales. Por lo que concatenamos la exacta (para que salga primero) con la lista que recibimos del diccionario.
Llamamos a Exporter con esa lista y escribimos en csv, json y html.

Si se eligio frase que contiene algunas palabras, el procedimiento es igual al anterior, pero en este caso, llama al metodo collect_products para que junte en una lista a todas las palabras que coinciden en una palabra, luego en dos, y asi hasta llegar a la cantidad de palabras totales que se introdujo por el usuario.

In [ ]:
    def __collect_products(self, products_list, amount):
        for x in amount:
            products_list += self.matching_words_to_product[x + 1]

Junta en una lista a todos los productos que contengan algunas de las palabras en su titulo.

  ### Clase Exporter:
        Se encarga de escribir en diferentes formatos los productos que se desea visualizar el precio más economico.
        Recibe los datos ordenados por Sorter.
        Es capaz de escribir en: .csv, .html, .json.

El constructor recibe la lista ordenada de productos, y ademas lo que ingreso el usuario para buscar, de manera que con eso el nombre del archivo es del estilo:

         input_dia-mes-año.formato

Tenemos 3 metodos publicos disponibles:
###### write_csv
Escribe un .csv con un csv.writer(), el cual escribe cada fila con un producto.

###### write_html
Escribe un html gracias a un string, el cual primero tiene el principio general de nuestro html con el framework de css bootstrap importado para una mejor estetica de la pagina. Luego recorre la lista de productos y escribe bloques de este estilo para cada producto:

            <tr>
                <th scope="row">Categoria</th>
                <td>
                    <a href="link">Titulo</a>
                </td>
                <td>Precio</td>
                <td>Fecha</td>
            </tr>
            
Concatena todos los bloques al string base, y despues de terminar de recorrer los productos, se concatena un string final para cerrar el html, y escribe el archivo con la cadena de caracteres final.

###### write_json 
Con cada producto que viene en el formato ["Page", "Category", "Title", "Price", "Link", "Time"], por ejemplo 
     
     ["Gezatek", "Auriculares", "Auriculares Samsung", "3055", "https://gezatek.com/auriculares-samsung", "20:27"]
Lo transforma en un diccionario del estilo:
    
    {"Page": "Gezatek", "Category": "Auriculares", "Title": "Auriculares Samsung", "Price": "3055", "Link": "https://gezatek.com/auriculares-samsung", "Time": "20:27" }

Esto se realiza para ser interpretado por el metodo dump que nos proporciona la libreria json.
Lo que hace json.dump es recibir un diccionario y escribirlo en formato json, asi que abrimos el nuevo archivo y escribimos con dump un diccionario final con clave "products" y valor una lista de diccionarios de cada producto del estilo anteriormente mostrado.

### SEARCHER (BACK)
    El modulo searcher es el encargado de recuperar los datos de la web, se divide a su vez en:
        1. Config: Configuracion de scrapy, con sus items, etc.
        2. Data: Archivos temporales y estaticos, los cuales son utilizados por las arañas para su funcionamiento.
        3. Spiders: Son las encargadas de recuperar la informacion de determinadas paginas, y exportarlas a csv temporales. Este modulo incluye su orchestrator, el cual las ejecuta.
        Un ejemplo podria ser la CompragamerSpider, las demas son similares.

    La primer parte de cada spider, se basa en su configuracion:
        1. name: Nombre de la araña.
        2. custom_settings: Forma de exportacion de los datos, en este caso csv.
        3. allowed_domains: Referencia al dominio en el cual se va a trabajar, del cual no se puede salir.
        4. item_count: Es una variable que utilizaremos mas adelante para limitar la salida de datos, y que el programa no tarde demasiado tiempo.
        5. start_urls: Links de las paginas desde donde se va a lanzar a recolectar datos, en este caso se toman desde un archivo temporal.
        6. rules: Reglas que la spider debe tener en cuenta en este caso, es que cada vez que encuentre un elemento de la lista (xpaths), entre a el dominio ejecutando parse_items,
            esto es muy util ya que al entrar a cada publicacion se puede extraer informacion que no se encuentra en la lista de la url incial. Otra regla podria ser que cada vez que encuentre
            un xpaths de un boton (por ejemplo siguiente pagina), siga esa url, para continuar con la busqueda.
        Por ultimo elimina el archivo temporal anterior, en el caso de que todavia siga existiendo (esto se hace si por alguna razon el programa se interrumpe en su proceso).

In [ ]:
    class CompragamerSpider(CrawlSpider):
        """
            Spider que recolecta datos de la pagina www.compragamer.com, con un limite de 15
        """

        name = 'compragamer'

        custom_settings = {'FEEDS': {'compragamer.csv': {'format': 'csv'}}}

        allowed_domains = ['compragamer.com']

        item_count = 0

        url = ""
        filePath = os.path.abspath("../TRZearcher/Back/Searcher/Data/pages_complete.txt")
        with open(filePath, "r") as pages:
            for page in pages:
                if page.find("compragamer") > 0:
                    url = page

        start_urls = [url]

        rules = {
            Rule(LinkExtractor(allow=(), restrict_xpaths=('//*[@class="products__item"]')),
                 callback='parse_item', follow=False)
        }

        try:
            os.remove('compragamer.csv')
        except OSError:
            pass

    La segunda parte de cada spider, se basa en la recuperacion de datos:
        - La llamada al TrzpidersItem se da para guardar los elementos que vamos a recuperar.
        - Cada elemento se recupera en mediante css (menos link y time), y dependiendo como se encuentre se arregla para su posterior salida en csv.
        - Los elementos a extraer por pagina se limitan a un maximo de 15 elementos, para que el programa no tarde demasiado tiempo y recupere muchos elementos.

In [ ]:
    def parse_item(self, response):
                """
                    Recolecta la informacion de cada item
                """

                item = TrzpidersItem()

                item['title'] = str(response.css(
                    '.filaNombre div::text').extract_first()).strip()

                item['price'] = str(response.css(
                    '.col-xs-5::text').extract_first()).split("$")[1].strip()

                item['category'] = str(response.css(
                    '.detalleNombre .product-card__name::text').extract_first()).capitalize().strip()

                item['link'] = str(response.url)

                item['time'] = (str(datetime.now().year) + "-" + str(datetime.now().month) + "-" + str(
                    datetime.now().day) + " " + str(datetime.now().hour) + ":" + str(datetime.now().minute))

                self.item_count += 1
                if self.item_count > 15:
                    raise CloseSpider('item exceeded')

                yield item

### TEST (BACK)
    Para testear las clases, se decidió conjuntamente no hacer las clásicas pruebas unitarias, ya que el código es complejo y no todos los métodos retornan un valor.
    El profesor asignado nos sugirió que la clase test heredara a la clase Sorter pero, por alguna razón, no permitía importar los casos de testeo.

    Para los métodos específicamente se suprimieron las partes que llamaban a otro objeto para simplificar el código, ya que lo se buscaba testear era que los métodos pasaran correctamente los filtros.
    Estas partes fueron reemplazadas por valores a retornar para poder ejecutar pruebas unitarias.

In [ ]:
    class TestSorter(unittest.TestCase):
        search_type = str

        def test_chooses_search_type_correctly(self):
            self.assertEqual(self.__define_search_type(1), "FRASE_EXACTA")
            self.assertEqual(self.__define_search_type(2), "CONTIENE_TODAS_LAS_PALABRAS")
            self.assertEqual(self.__define_search_type(3), "CONTIENE_ALGUNAS_PALABRAS")

        def test_normalizes_price(self):
            self.assertEqual(self.__normalize_price("$21.00"), '$21')

        def test_exports_correctly(self):
            self.search_type = "FRASE_EXACTA"
            self.assertEqual(self.__export(), 'Exportado frase exacta')
            self.search_type = "CONTIENE_TODAS_LAS_PALABRAS"
            self.assertEqual(self.__export(), 'Exportado contiene todas las palabras')
            self.search_type = "CONTIENE_ALGUNAS_PALABRAS"
            self.assertEqual(self.__export(), 'Exportado contiene algunas palabras')

        def test_sort_by_price(self):
            products_list = []
            with open("testpage.csv", 'r') as f:
                file = csv.DictReader(f, delimiter=",")

                for line in file:
                    product = [line["Page"], line["Category"].lower(), line["Title"].lower(), int(line["Price"]), line["Link"],
                               line["Time"]]
                    products_list.append(product)
            products_list_sorted = products_list.sort(key=lambda e: e[3])
            self.assertEqual(self.__sort_by_price(products_list), products_list_sorted)



        def __sort_by_price(self, products_list):
            products_list.sort(key=lambda e: e[3])
        def __export(self):

            if self.search_type == "FRASE_EXACTA":
                return 'Exportado frase exacta'

            elif self.search_type == "CONTIENE_TODAS_LAS_PALABRAS":
                return 'Exportado contiene todas las palabras'

            elif self.search_type == "CONTIENE_ALGUNAS_PALABRAS":
                return 'Exportado contiene algunas palabras'
        def __normalize_price(self, price):
            price = price.replace('"', "")
            price = price.replace(',', "")
            price = price.replace('.00', "")
            price = price.replace('.', "")
            return price
        def __define_search_type(self,search_type):
            if int(search_type) == 1:
                return "FRASE_EXACTA"
            if int(search_type) == 2:
                return "CONTIENE_TODAS_LAS_PALABRAS"
            if int(search_type) == 3:
                return "CONTIENE_ALGUNAS_PALABRAS"